In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import os
import glob

# initial parameters
epochs = 100
lr = 1e-3
batch_size = 64
img_dims = (96,96,3)

data = []
labels = []

# load image files from the dataset
image_files = [f for f in glob.glob(r'C:\Users\shubham\Research work pyhton\GenderC\gender_dataset_face' + "/**/*", recursive=True) if not os.path.isdir(f)]
random.shuffle(image_files)

# converting images to arrays and labelling the categories
for img in image_files:

    image = cv2.imread(img)
    
    image = cv2.resize(image, (img_dims[0],img_dims[1]))
    image = img_to_array(image)
    data.append(image)

    label = img.split(os.path.sep)[-2] # C:\Files\gender_dataset_face\woman\face_1162.jpg
    if label == "woman":
        label = 1
    else:
        label = 0
        
    labels.append([label]) # [[1], [0], [0], ...]

# pre-processing
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

# split dataset for training and validation
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2,
                                                  random_state=42)

trainY = to_categorical(trainY, num_classes=2) # [[1, 0], [0, 1], [0, 1], ...]
testY = to_categorical(testY, num_classes=2)

# augmenting datset 
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                         horizontal_flip=True, fill_mode="nearest")

# define model
def build(width, height, depth, classes):
    model = Sequential()
    inputShape = (height, width, depth)
    chanDim = -1

    if K.image_data_format() == "channels_first": #Returns a string, either 'channels_first' or 'channels_last'
        inputShape = (depth, height, width)
        chanDim = 1
    
    # The axis that should be normalized, after a Conv2D layer with data_format="channels_first", 
    # set axis=1 in BatchNormalization.

    model.add(Conv2D(32, (3,3), padding="same", input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))

    model.add(Conv2D(64, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))

    model.add(Conv2D(128, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(classes))
    model.add(Activation("sigmoid"))

    return model

# build model
model = build(width=img_dims[0], height=img_dims[1], depth=img_dims[2],
                            classes=2)

# compile the model
opt = Adam(lr=lr, decay=lr/epochs)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# train the model
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=batch_size),
                        validation_data=(testX,testY),
                        steps_per_epoch=len(trainX) // batch_size,
                        epochs=epochs, verbose=1)

# save the model to disk
model.save('gender_detection.model')

# plot training/validation loss/accuracy
plt.style.use("ggplot")
plt.figure()
N = epochs
plt.plot(np.arange(0,N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0,N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0,N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0,N), H.history["val_accuracy"], label="val_acc")

plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper right")

# save plot to disk
plt.savefig('plot.png')

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
28/28 [==============================] - 25s 892ms/step - loss: 0.7369 - accuracy: 0.7423 - val_loss: 0.7053 - val_accuracy: 0.5346
Epoch 2/100
28/28 [==============================] - 24s 847ms/step - loss: 0.4869 - accuracy: 0.8074 - val_loss: 0.6888 - val_accuracy: 0.5866
Epoch 3/100
28/28 [==============================] - 22s 799ms/step - loss: 0.3935 - accuracy: 0.8456 - val_loss: 0.7081 - val_accuracy: 0.5346
Epoch 4/100
28/28 [==============================] - 23s 815ms/step - loss: 0.3756 - accuracy: 0.8535 - val_loss: 0.7800 - val_accuracy: 0.5758
Epoch 5/100
28/28 [==============================] - 24s 865ms/step - loss: 0.3592 - accuracy: 0.8596 - val_loss: 0.8096 - val_accuracy: 0.5584
Epoch 6/100
28/28 [==============================] - 21s 768ms/step - loss: 0.3194 - accuracy: 0.8804 - val_loss: 1.2049 - val_accuracy: 0.4675
Epoch 7/100
28/28 [==============================] - 23s 813

Epoch 56/100
28/28 [==============================] - 22s 790ms/step - loss: 0.0610 - accuracy: 0.9798 - val_loss: 0.0855 - val_accuracy: 0.9675
Epoch 57/100
28/28 [==============================] - 22s 785ms/step - loss: 0.0728 - accuracy: 0.9736 - val_loss: 0.1288 - val_accuracy: 0.9589
Epoch 58/100
28/28 [==============================] - 22s 792ms/step - loss: 0.0679 - accuracy: 0.9764 - val_loss: 0.1999 - val_accuracy: 0.9502
Epoch 59/100
28/28 [==============================] - 22s 785ms/step - loss: 0.0632 - accuracy: 0.9759 - val_loss: 1.2044 - val_accuracy: 0.7446
Epoch 60/100
28/28 [==============================] - 23s 817ms/step - loss: 0.0818 - accuracy: 0.9688 - val_loss: 0.0804 - val_accuracy: 0.9719
Epoch 61/100
28/28 [==============================] - 22s 785ms/step - loss: 0.0620 - accuracy: 0.9753 - val_loss: 0.3523 - val_accuracy: 0.9026
Epoch 62/100
28/28 [==============================] - 22s 785ms/step - loss: 0.0800 - accuracy: 0.9680 - val_loss: 0.1232 - val_ac

FailedPreconditionError: gender_detection.model is not a directory